In [3]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
import pandas as pd
#import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import re
import warnings
import math
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
dataset = pd.read_csv("fake_news_1.txt", sep='\t',engine='python')


In [5]:
dataset.head(2)

,id,news,class
0,1,The U.S. CDC quietly updated their numbers in ...,False
1,2,Social media posts shared thousands of times a...,False


In [6]:
from detoxify import Detoxify
import numpy as np
import pandas as pd

In [7]:
predictor = Detoxify('multilingual')

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at None and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
predictor.predict('Mobile towers cause covid infection')

{'toxicity': 0.0011730007,
 'severe_toxicity': 0.001023982,
 'obscene': 0.006120092,
 'identity_attack': 0.0015339844,
 'insult': 0.004656217,
 'threat': 0.00081497297,
 'sexual_explicit': 0.0002187606}

In [9]:
sentences = [
        'You suck!',
        'How much is this bag?',
        'Why do you always get to be an jerk?',
        'I bought this gift for my mother',
        'I will hurt you'
]

In [10]:
for sentence in sentences:
    results = predictor.predict(sentence)
    print (results)

{'toxicity': 0.99660456, 'severe_toxicity': 0.012938024, 'obscene': 0.57545424, 'identity_attack': 0.0018781582, 'insult': 0.9799171, 'threat': 0.004116688, 'sexual_explicit': 0.08039768}
{'toxicity': 0.0016920753, 'severe_toxicity': 1.4579835e-05, 'obscene': 0.00018599293, 'identity_attack': 7.167008e-05, 'insult': 0.00056396285, 'threat': 3.5181823e-05, 'sexual_explicit': 2.9745019e-05}
{'toxicity': 0.9947636, 'severe_toxicity': 0.00069288764, 'obscene': 0.043506645, 'identity_attack': 0.004385927, 'insult': 0.9752337, 'threat': 0.0015983073, 'sexual_explicit': 0.0012396235}
{'toxicity': 0.00089999806, 'severe_toxicity': 1.5985268e-05, 'obscene': 0.000108549284, 'identity_attack': 0.00012365112, 'insult': 0.00023804733, 'threat': 3.0528885e-05, 'sexual_explicit': 2.7818343e-05}
{'toxicity': 0.931444, 'severe_toxicity': 0.007850298, 'obscene': 0.04203803, 'identity_attack': 0.0037544132, 'insult': 0.036925927, 'threat': 0.7674866, 'sexual_explicit': 0.017453792}


In [11]:
print(dataset.columns)

Index(['id', 'news', 'class'], dtype='object')


In [12]:
dataset.head()

,id,news,class
0,1,The U.S. CDC quietly updated their numbers in ...,False
1,2,Social media posts shared thousands of times a...,False
2,3,The CDC updated the COVID-19 number to admit t...,False
3,4,Either masks work or they donâ€™t; if masks wo...,False
4,5,The RT-PCR test for the virus that causes COVI...,False


In [13]:
import xlrd
 
# Give the location of the file
loc = (r'fake_news_11.xlsx')
 
# To open Workbook
wb = xlrd.open_workbook(loc)
sheet = wb.sheet_by_index(0)
 
# For row 0 and column 0
print(sheet.cell_value(0, 0))

id


In [14]:
wb = xlrd.open_workbook(loc)
sheet = wb.sheet_by_index(0)
#sheet.cell_value(0, 0)
 
for i in range(sheet.nrows):
    print(sheet.cell_value(i, 1))
    #for j in range(sheet.ncols):
        

news
The U.S. CDC quietly updated their numbers in August to show that only 6% of all coronavirus deaths were completely due to the coronavirus alone
Social media posts shared thousands of times and retweeted by President Donald Trump claim the U.S Centers for Disease Control and Prevention cut the national COVID-19 death toll by 94%, citing a CDC comorbidity chart.
The CDC updated the COVID-19 number to admit that only 6% of all the 153,504 deaths recorded actually died from COVID-19.
Either masks work or they donâ€™t; if masks work, then physical distancing and lockdowns are unnecessar
The RT-PCR test for the virus that causes COVID-19 detects human DNA on chromosome 8, therefore all tests will give a positive result
The Governor of Maine has ordered restaurant staff to wear dog cone-style face visors as a precautionary measure against COVID-19.
The greatest trick the U.S. CDC ever pulled was convincing the world you can be sick without having symptoms.
The survival rate of COVID-19 

All elective or non-emergency surgeries are banned to prioritize coronavirus treatments and prevent the spread.
U.S. House Speaker Nancy Pelosi was in Wuhan China in time to release the novel coronavirus and sabotage the U.S.-China trade deal.
U.S. House Speaker Nancy Pelosi was in Wuhan China six days after the impeachment proceedings against President Trump ended.
Nearly half of (Missouri) counties have not reported positive (COVID-19) cases.
"Spraying chlorine or alcohol on the skin kills viruses in the body"
"Only older adults and young people are at risk"
"Children cannot get COVID-19"
"COVID-19 is just like the flu"
"Everyone with COVID-19 dies"
"Cats and dogs spread coronavirus"
"Face masks always protect against coronavirus"
"Hand dryers kill coronavirus"
"SARS-CoV-2 is just a mutated form of the common cold"
"You have to be with someone for 10 minutes to catch the virus"
"Rinsing the nose with saline protects against coronavirus"
"You can protect yourself by gargling bleach"

In [15]:
sheet.cell_value(1, 1)

'The U.S. CDC quietly updated their numbers in August to show that only 6% of all coronavirus deaths were completely due to the coronavirus alone'

In [16]:
results = predictor.predict(sheet.cell_value(0, 1))
print (results)

{'toxicity': 0.00029946756, 'severe_toxicity': 6.56572e-05, 'obscene': 0.00045753963, 'identity_attack': 0.00016274168, 'insult': 0.0004023502, 'threat': 6.788591e-05, 'sexual_explicit': 4.1863645e-05}


In [42]:
i=1
for i in range(sheet.nrows):
    results = predictor.predict(sheet.cell_value(i, 1))
    print (results)

{'toxicity': 0.00029946756, 'severe_toxicity': 6.56572e-05, 'obscene': 0.00045753963, 'identity_attack': 0.00016274168, 'insult': 0.0004023502, 'threat': 6.788591e-05, 'sexual_explicit': 4.1863645e-05}
{'toxicity': 0.00042616294, 'severe_toxicity': 3.562209e-05, 'obscene': 0.00015766341, 'identity_attack': 0.0001237056, 'insult': 0.00014089518, 'threat': 4.9529368e-05, 'sexual_explicit': 3.1786876e-05}
{'toxicity': 0.00049101555, 'severe_toxicity': 2.0898462e-05, 'obscene': 0.00011578303, 'identity_attack': 7.492199e-05, 'insult': 0.00017742712, 'threat': 3.276651e-05, 'sexual_explicit': 2.1647547e-05}
{'toxicity': 0.00035796335, 'severe_toxicity': 3.1277374e-05, 'obscene': 0.00016451564, 'identity_attack': 9.2840084e-05, 'insult': 0.00016736524, 'threat': 4.161154e-05, 'sexual_explicit': 2.6008423e-05}
{'toxicity': 0.00062728045, 'severe_toxicity': 3.971006e-05, 'obscene': 0.00020694153, 'identity_attack': 0.00015412776, 'insult': 0.00017084215, 'threat': 9.670234e-05, 'sexual_explici

{'toxicity': 0.0004474278, 'severe_toxicity': 2.4801377e-05, 'obscene': 0.00017923293, 'identity_attack': 9.0468006e-05, 'insult': 0.00025775807, 'threat': 2.8939188e-05, 'sexual_explicit': 2.2815591e-05}
{'toxicity': 0.0010704431, 'severe_toxicity': 3.115107e-05, 'obscene': 0.00011512215, 'identity_attack': 0.0001656196, 'insult': 0.00016959728, 'threat': 0.00015999566, 'sexual_explicit': 3.5907906e-05}
{'toxicity': 0.00044442693, 'severe_toxicity': 2.1309277e-05, 'obscene': 0.0001145549, 'identity_attack': 9.488477e-05, 'insult': 0.00016796443, 'threat': 3.9946877e-05, 'sexual_explicit': 2.2556584e-05}
{'toxicity': 0.0008140749, 'severe_toxicity': 5.2998406e-05, 'obscene': 0.00032211328, 'identity_attack': 0.00016301362, 'insult': 0.0002482578, 'threat': 0.00011640717, 'sexual_explicit': 4.5846813e-05}
{'toxicity': 0.0019644697, 'severe_toxicity': 5.4254484e-05, 'obscene': 0.00021556781, 'identity_attack': 0.00018537186, 'insult': 0.00036820947, 'threat': 0.0004458954, 'sexual_explic

{'toxicity': 0.0007081007, 'severe_toxicity': 2.1263928e-05, 'obscene': 0.00020770774, 'identity_attack': 7.438891e-05, 'insult': 0.0002607243, 'threat': 3.243331e-05, 'sexual_explicit': 2.9140216e-05}
{'toxicity': 0.0003465819, 'severe_toxicity': 2.883456e-05, 'obscene': 0.00015662845, 'identity_attack': 8.51566e-05, 'insult': 0.0002091649, 'threat': 3.87433e-05, 'sexual_explicit': 2.4501875e-05}
{'toxicity': 0.30835345, 'severe_toxicity': 0.00025367705, 'obscene': 0.0017219956, 'identity_attack': 0.0007687697, 'insult': 0.0636802, 'threat': 0.061091103, 'sexual_explicit': 0.0007864472}
{'toxicity': 0.00038308598, 'severe_toxicity': 2.2038656e-05, 'obscene': 0.00010017084, 'identity_attack': 0.000107284584, 'insult': 0.00014379581, 'threat': 3.826252e-05, 'sexual_explicit': 2.2176728e-05}
{'toxicity': 0.00050615735, 'severe_toxicity': 2.9123239e-05, 'obscene': 0.00016347296, 'identity_attack': 0.00012926558, 'insult': 0.00018289563, 'threat': 5.029702e-05, 'sexual_explicit': 2.7867307

{'toxicity': 0.00030032246, 'severe_toxicity': 3.2011223e-05, 'obscene': 0.0001674754, 'identity_attack': 8.320783e-05, 'insult': 0.00017083777, 'threat': 4.3834454e-05, 'sexual_explicit': 3.0415204e-05}
{'toxicity': 0.0069308816, 'severe_toxicity': 0.00026230284, 'obscene': 0.0005552423, 'identity_attack': 0.0024908646, 'insult': 0.0011797161, 'threat': 0.0019425628, 'sexual_explicit': 0.00041479585}
{'toxicity': 0.013887613, 'severe_toxicity': 3.3983863e-05, 'obscene': 0.00020068264, 'identity_attack': 0.0012644166, 'insult': 0.0008738592, 'threat': 0.00024322844, 'sexual_explicit': 0.0004997241}
{'toxicity': 0.47202584, 'severe_toxicity': 0.0009793984, 'obscene': 0.0012002782, 'identity_attack': 0.009806006, 'insult': 0.003347261, 'threat': 0.41777825, 'sexual_explicit': 0.0016203283}
{'toxicity': 0.001196436, 'severe_toxicity': 7.002646e-05, 'obscene': 0.0005181155, 'identity_attack': 0.00030799754, 'insult': 0.00037578557, 'threat': 0.00016046777, 'sexual_explicit': 4.6805082e-05}

{'toxicity': 0.0007482204, 'severe_toxicity': 7.690482e-05, 'obscene': 0.0003785735, 'identity_attack': 0.0001581514, 'insult': 0.00026995386, 'threat': 0.00011790301, 'sexual_explicit': 9.434775e-05}
{'toxicity': 0.0007059732, 'severe_toxicity': 3.329887e-05, 'obscene': 0.0002597928, 'identity_attack': 0.00010675473, 'insult': 0.0003138333, 'threat': 6.869353e-05, 'sexual_explicit': 4.4777174e-05}
{'toxicity': 0.00075761526, 'severe_toxicity': 1.6487731e-05, 'obscene': 0.0001216242, 'identity_attack': 9.645947e-05, 'insult': 0.00030801664, 'threat': 3.157322e-05, 'sexual_explicit': 1.946418e-05}
{'toxicity': 0.0005494111, 'severe_toxicity': 2.2633165e-05, 'obscene': 0.00016537438, 'identity_attack': 9.3581555e-05, 'insult': 0.0002190152, 'threat': 3.4436056e-05, 'sexual_explicit': 2.1654856e-05}
{'toxicity': 0.0012773302, 'severe_toxicity': 1.7085025e-05, 'obscene': 0.00010505985, 'identity_attack': 0.00010569974, 'insult': 0.00044501282, 'threat': 4.2114352e-05, 'sexual_explicit': 2.

...

{'toxicity': 0.0016617965, 'severe_toxicity': 2.441159e-05, 'obscene': 0.00015899345, 'identity_attack': 0.00019825564, 'insult': 0.00022300116, 'threat': 7.569319e-05, 'sexual_explicit': 4.47706e-05}
{'toxicity': 0.0004869646, 'severe_toxicity': 1.8827273e-05, 'obscene': 0.0001321798, 'identity_attack': 7.7646066e-05, 'insult': 0.0002292222, 'threat': 2.984202e-05, 'sexual_explicit': 2.0227932e-05}
{'toxicity': 0.0016876422, 'severe_toxicity': 2.2224767e-05, 'obscene': 0.00016019685, 'identity_attack': 9.894729e-05, 'insult': 0.00029437616, 'threat': 0.0001076841, 'sexual_explicit': 3.931442e-05}
{'toxicity': 0.03667468, 'severe_toxicity': 0.00040548758, 'obscene': 0.00071612897, 'identity_attack': 0.038294993, 'insult': 0.0019226759, 'threat': 0.0005010042, 'sexual_explicit': 0.00026713213}
{'toxicity': 0.0007347374, 'severe_toxicity': 1.5452291e-05, 'obscene': 9.896429e-05, 'identity_attack': 0.00010545119, 'insult': 0.00024899904, 'threat': 2.7562883e-05, 'sexual_explicit': 1.70292

{'toxicity': 0.05352612, 'severe_toxicity': 2.8010785e-05, 'obscene': 0.0004400977, 'identity_attack': 0.0018338148, 'insult': 0.02473573, 'threat': 9.657277e-05, 'sexual_explicit': 9.7729295e-05}
{'toxicity': 0.00059518154, 'severe_toxicity': 2.27341e-05, 'obscene': 0.00016552299, 'identity_attack': 9.0051595e-05, 'insult': 0.00027523056, 'threat': 2.8971192e-05, 'sexual_explicit': 2.2151278e-05}
{'toxicity': 0.00097301754, 'severe_toxicity': 1.9588217e-05, 'obscene': 0.00018618474, 'identity_attack': 9.535043e-05, 'insult': 0.0003748812, 'threat': 3.743249e-05, 'sexual_explicit': 2.4604857e-05}
{'toxicity': 0.0007894826, 'severe_toxicity': 8.0546015e-05, 'obscene': 0.00056781474, 'identity_attack': 0.00020065471, 'insult': 0.0004177952, 'threat': 0.0001501479, 'sexual_explicit': 5.6452853e-05}
{'toxicity': 0.0003388586, 'severe_toxicity': 3.427421e-05, 'obscene': 0.00015641919, 'identity_attack': 9.863717e-05, 'insult': 0.00016785701, 'threat': 5.206604e-05, 'sexual_explicit': 3.0208

{'toxicity': 0.00039838205, 'severe_toxicity': 2.1341268e-05, 'obscene': 0.00012080794, 'identity_attack': 8.9104455e-05, 'insult': 0.0001810911, 'threat': 3.581909e-05, 'sexual_explicit': 2.0977255e-05}
{'toxicity': 0.0005647027, 'severe_toxicity': 2.2374714e-05, 'obscene': 0.00012245942, 'identity_attack': 9.0397974e-05, 'insult': 0.0002075007, 'threat': 2.7032103e-05, 'sexual_explicit': 3.0611565e-05}
{'toxicity': 0.0008543179, 'severe_toxicity': 2.128996e-05, 'obscene': 0.00014935456, 'identity_attack': 0.00014170453, 'insult': 0.0003097113, 'threat': 3.580017e-05, 'sexual_explicit': 1.8903394e-05}
{'toxicity': 0.037897095, 'severe_toxicity': 2.247246e-05, 'obscene': 0.0002046095, 'identity_attack': 0.018181818, 'insult': 0.0030315884, 'threat': 0.0001871944, 'sexual_explicit': 6.216338e-05}
{'toxicity': 0.0046881437, 'severe_toxicity': 3.092573e-05, 'obscene': 0.00016329816, 'identity_attack': 0.00045148318, 'insult': 0.00049639423, 'threat': 0.0005565271, 'sexual_explicit': 7.658

{'toxicity': 0.001613479, 'severe_toxicity': 3.5179073e-05, 'obscene': 0.00021825517, 'identity_attack': 0.00016028776, 'insult': 0.00043289614, 'threat': 0.00013639223, 'sexual_explicit': 3.46477e-05}
{'toxicity': 0.0008848162, 'severe_toxicity': 1.7028062e-05, 'obscene': 0.000104334336, 'identity_attack': 7.743695e-05, 'insult': 0.00030507336, 'threat': 4.482212e-05, 'sexual_explicit': 2.5051613e-05}
{'toxicity': 0.00074124907, 'severe_toxicity': 1.8789877e-05, 'obscene': 0.00012381075, 'identity_attack': 9.395245e-05, 'insult': 0.00023978445, 'threat': 3.6229063e-05, 'sexual_explicit': 2.0919259e-05}
{'toxicity': 0.000645199, 'severe_toxicity': 1.9822191e-05, 'obscene': 0.00010548709, 'identity_attack': 0.00010887781, 'insult': 0.00021966122, 'threat': 3.207304e-05, 'sexual_explicit': 2.5324704e-05}
{'toxicity': 0.0006077147, 'severe_toxicity': 2.5619207e-05, 'obscene': 0.00013707006, 'identity_attack': 0.00011421307, 'insult': 0.00021453215, 'threat': 5.1701143e-05, 'sexual_explici

{'toxicity': 0.0016534871, 'severe_toxicity': 3.454898e-05, 'obscene': 0.00020210381, 'identity_attack': 0.00017020656, 'insult': 0.00026085236, 'threat': 0.00013383092, 'sexual_explicit': 0.00011160572}
{'toxicity': 0.0028696188, 'severe_toxicity': 5.6367e-05, 'obscene': 0.00016150405, 'identity_attack': 0.00019597793, 'insult': 0.00027306768, 'threat': 0.00080287625, 'sexual_explicit': 5.533178e-05}
{'toxicity': 0.0024105844, 'severe_toxicity': 2.0859596e-05, 'obscene': 0.00016932451, 'identity_attack': 0.00015985704, 'insult': 0.0006549481, 'threat': 3.4929493e-05, 'sexual_explicit': 2.8932925e-05}
{'toxicity': 0.0006098319, 'severe_toxicity': 5.0254162e-05, 'obscene': 0.00028862213, 'identity_attack': 9.329144e-05, 'insult': 0.0003176932, 'threat': 0.00013342052, 'sexual_explicit': 6.860901e-05}
{'toxicity': 0.000688801, 'severe_toxicity': 3.2787768e-05, 'obscene': 0.000140009, 'identity_attack': 0.0003338553, 'insult': 0.0001914481, 'threat': 6.520819e-05, 'sexual_explicit': 2.539

In [41]:
i=1
for i in range(sheet.nrows):
    results = predictor.predict(sheet.cell_value(i, 1))
    #print (results)
    for key in results.items():
        for key, value in results.items():
            print("\t%s"%value)
            
print("_________________Toxicity scores calculated Successfully!__________________")    
type(results)

	0.00029946756
	6.56572e-05
	0.00045753963
	0.00016274168
	0.0004023502
	6.788591e-05
	4.1863645e-05
	0.00029946756
	6.56572e-05
	0.00045753963
	0.00016274168
	0.0004023502
	6.788591e-05
	4.1863645e-05
	0.00029946756
	6.56572e-05
	0.00045753963
	0.00016274168
	0.0004023502
	6.788591e-05
	4.1863645e-05
	0.00029946756
	6.56572e-05
	0.00045753963
	0.00016274168
	0.0004023502
	6.788591e-05
	4.1863645e-05
	0.00029946756
	6.56572e-05
	0.00045753963
	0.00016274168
	0.0004023502
	6.788591e-05
	4.1863645e-05
	0.00029946756
	6.56572e-05
	0.00045753963
	0.00016274168
	0.0004023502
	6.788591e-05
	4.1863645e-05
	0.00029946756
	6.56572e-05
	0.00045753963
	0.00016274168
	0.0004023502
	6.788591e-05
	4.1863645e-05
	0.00042616294
	3.562209e-05
	0.00015766341
	0.0001237056
	0.00014089518
	4.9529368e-05
	3.1786876e-05
	0.00042616294
	3.562209e-05
	0.00015766341
	0.0001237056
	0.00014089518
	4.9529368e-05
	3.1786876e-05
	0.00042616294
	3.562209e-05
	0.00015766341
	0.0001237056
	0.00014089518
	4.9529368e-05

	0.0010607126
	4.6269957e-05
	0.00036239033
	0.00031135883
	0.0006120355
	7.616019e-05
	3.3442844e-05
	0.0010607126
	4.6269957e-05
	0.00036239033
	0.00031135883
	0.0006120355
	7.616019e-05
	3.3442844e-05
	0.0010607126
	4.6269957e-05
	0.00036239033
	0.00031135883
	0.0006120355
	7.616019e-05
	3.3442844e-05
	0.0010607126
	4.6269957e-05
	0.00036239033
	0.00031135883
	0.0006120355
	7.616019e-05
	3.3442844e-05
	0.0010607126
	4.6269957e-05
	0.00036239033
	0.00031135883
	0.0006120355
	7.616019e-05
	3.3442844e-05
	0.0010607126
	4.6269957e-05
	0.00036239033
	0.00031135883
	0.0006120355
	7.616019e-05
	3.3442844e-05
	0.0010607126
	4.6269957e-05
	0.00036239033
	0.00031135883
	0.0006120355
	7.616019e-05
	3.3442844e-05
	0.00033380312
	3.6905985e-05
	0.00017632685
	0.000106584666
	0.00015788035
	4.7272497e-05
	3.214261e-05
	0.00033380312
	3.6905985e-05
	0.00017632685
	0.000106584666
	0.00015788035
	4.7272497e-05
	3.214261e-05
	0.00033380312
	3.6905985e-05
	0.00017632685
	0.000106584666
	0.00015788035


	0.0004597665
	6.9224014e-05
	0.0004054267
	0.00016442155
	0.0006593368
	7.587299e-05
	5.9436185e-05
	0.0004597665
	6.9224014e-05
	0.0004054267
	0.00016442155
	0.0006593368
	7.587299e-05
	5.9436185e-05
	0.0004597665
	6.9224014e-05
	0.0004054267
	0.00016442155
	0.0006593368
	7.587299e-05
	5.9436185e-05
	0.0004597665
	6.9224014e-05
	0.0004054267
	0.00016442155
	0.0006593368
	7.587299e-05
	5.9436185e-05
	0.0004597665
	6.9224014e-05
	0.0004054267
	0.00016442155
	0.0006593368
	7.587299e-05
	5.9436185e-05
	0.0004597665
	6.9224014e-05
	0.0004054267
	0.00016442155
	0.0006593368
	7.587299e-05
	5.9436185e-05
	0.0004597665
	6.9224014e-05
	0.0004054267
	0.00016442155
	0.0006593368
	7.587299e-05
	5.9436185e-05
	0.00055691105
	1.6557062e-05
	0.00010709395
	9.0546884e-05
	0.00021175147
	2.5962456e-05
	1.892605e-05
	0.00055691105
	1.6557062e-05
	0.00010709395
	9.0546884e-05
	0.00021175147
	2.5962456e-05
	1.892605e-05
	0.00055691105
	1.6557062e-05
	0.00010709395
	9.0546884e-05
	0.00021175147
	2.5962456

	0.0004999988
	1.9224673e-05
	0.00014057472
	7.124984e-05
	0.0002128143
	2.7295187e-05
	2.441797e-05
	0.0004999988
	1.9224673e-05
	0.00014057472
	7.124984e-05
	0.0002128143
	2.7295187e-05
	2.441797e-05
	0.0004999988
	1.9224673e-05
	0.00014057472
	7.124984e-05
	0.0002128143
	2.7295187e-05
	2.441797e-05
	0.0004999988
	1.9224673e-05
	0.00014057472
	7.124984e-05
	0.0002128143
	2.7295187e-05
	2.441797e-05
	0.0004999988
	1.9224673e-05
	0.00014057472
	7.124984e-05
	0.0002128143
	2.7295187e-05
	2.441797e-05
	0.0004999988
	1.9224673e-05
	0.00014057472
	7.124984e-05
	0.0002128143
	2.7295187e-05
	2.441797e-05
	0.0004999988
	1.9224673e-05
	0.00014057472
	7.124984e-05
	0.0002128143
	2.7295187e-05
	2.441797e-05
	0.003305805
	5.273413e-05
	0.0002466198
	0.0004029634
	0.00017396877
	0.00034518688
	0.00034252243
	0.003305805
	5.273413e-05
	0.0002466198
	0.0004029634
	0.00017396877
	0.00034518688
	0.00034252243
	0.003305805
	5.273413e-05
	0.0002466198
	0.0004029634
	0.00017396877
	0.00034518688
	0.00034

	0.0003797589
	4.9625585e-05
	0.00025382076
	0.000119404736
	0.00023391425
	7.112567e-05
	3.4344717e-05
	0.0003797589
	4.9625585e-05
	0.00025382076
	0.000119404736
	0.00023391425
	7.112567e-05
	3.4344717e-05
	0.0003797589
	4.9625585e-05
	0.00025382076
	0.000119404736
	0.00023391425
	7.112567e-05
	3.4344717e-05
	0.0003797589
	4.9625585e-05
	0.00025382076
	0.000119404736
	0.00023391425
	7.112567e-05
	3.4344717e-05
	0.0003797589
	4.9625585e-05
	0.00025382076
	0.000119404736
	0.00023391425
	7.112567e-05
	3.4344717e-05
	0.0003797589
	4.9625585e-05
	0.00025382076
	0.000119404736
	0.00023391425
	7.112567e-05
	3.4344717e-05
	0.0003797589
	4.9625585e-05
	0.00025382076
	0.000119404736
	0.00023391425
	7.112567e-05
	3.4344717e-05
	0.00074333156
	5.5202072e-05
	0.00025693906
	0.0001111802
	0.0002479869
	0.00011567598
	6.951508e-05
	0.00074333156
	5.5202072e-05
	0.00025693906
	0.0001111802
	0.0002479869
	0.00011567598
	6.951508e-05
	0.00074333156
	5.5202072e-05
	0.00025693906
	0.0001111802
	0.000247

...

	0.05122498
	9.7079115e-05
	0.00020885242
	0.0008440016
	0.0012998328
	0.016092436
	0.00041077865
	0.05122498
	9.7079115e-05
	0.00020885242
	0.0008440016
	0.0012998328
	0.016092436
	0.00041077865
	0.05122498
	9.7079115e-05
	0.00020885242
	0.0008440016
	0.0012998328
	0.016092436
	0.00041077865
	0.05122498
	9.7079115e-05
	0.00020885242
	0.0008440016
	0.0012998328
	0.016092436
	0.00041077865
	0.05122498
	9.7079115e-05
	0.00020885242
	0.0008440016
	0.0012998328
	0.016092436
	0.00041077865
	0.05122498
	9.7079115e-05
	0.00020885242
	0.0008440016
	0.0012998328
	0.016092436
	0.00041077865
	0.05122498
	9.7079115e-05
	0.00020885242
	0.0008440016
	0.0012998328
	0.016092436
	0.00041077865
	0.00046854734
	2.535701e-05
	0.00014913098
	8.857887e-05
	0.00020836311
	3.908948e-05
	2.4203773e-05
	0.00046854734
	2.535701e-05
	0.00014913098
	8.857887e-05
	0.00020836311
	3.908948e-05
	2.4203773e-05
	0.00046854734
	2.535701e-05
	0.00014913098
	8.857887e-05
	0.00020836311
	3.908948e-05
	2.4203773e-05
	0.00046

	0.00091638963
	1.5987524e-05
	0.00014387316
	7.11214e-05
	0.00034585476
	2.2640203e-05
	2.502418e-05
	0.00091638963
	1.5987524e-05
	0.00014387316
	7.11214e-05
	0.00034585476
	2.2640203e-05
	2.502418e-05
	0.00091638963
	1.5987524e-05
	0.00014387316
	7.11214e-05
	0.00034585476
	2.2640203e-05
	2.502418e-05
	0.00091638963
	1.5987524e-05
	0.00014387316
	7.11214e-05
	0.00034585476
	2.2640203e-05
	2.502418e-05
	0.00091638963
	1.5987524e-05
	0.00014387316
	7.11214e-05
	0.00034585476
	2.2640203e-05
	2.502418e-05
	0.00091638963
	1.5987524e-05
	0.00014387316
	7.11214e-05
	0.00034585476
	2.2640203e-05
	2.502418e-05
	0.00091638963
	1.5987524e-05
	0.00014387316
	7.11214e-05
	0.00034585476
	2.2640203e-05
	2.502418e-05
	0.00050875795
	2.2086162e-05
	0.00015116988
	9.2194816e-05
	0.00024028546
	3.3137345e-05
	2.5467716e-05
	0.00050875795
	2.2086162e-05
	0.00015116988
	9.2194816e-05
	0.00024028546
	3.3137345e-05
	2.5467716e-05
	0.00050875795
	2.2086162e-05
	0.00015116988
	9.2194816e-05
	0.00024028546
	

	0.0019760618
	2.4995963e-05
	0.00019430726
	0.00012270476
	0.00078204175
	5.4010827e-05
	2.920969e-05
	0.0019760618
	2.4995963e-05
	0.00019430726
	0.00012270476
	0.00078204175
	5.4010827e-05
	2.920969e-05
	0.0019760618
	2.4995963e-05
	0.00019430726
	0.00012270476
	0.00078204175
	5.4010827e-05
	2.920969e-05
	0.0019760618
	2.4995963e-05
	0.00019430726
	0.00012270476
	0.00078204175
	5.4010827e-05
	2.920969e-05
	0.0019760618
	2.4995963e-05
	0.00019430726
	0.00012270476
	0.00078204175
	5.4010827e-05
	2.920969e-05
	0.0019760618
	2.4995963e-05
	0.00019430726
	0.00012270476
	0.00078204175
	5.4010827e-05
	2.920969e-05
	0.0019760618
	2.4995963e-05
	0.00019430726
	0.00012270476
	0.00078204175
	5.4010827e-05
	2.920969e-05
	0.0021645825
	7.251305e-05
	0.00023142833
	0.00022225524
	0.00027056944
	0.00095202215
	6.872898e-05
	0.0021645825
	7.251305e-05
	0.00023142833
	0.00022225524
	0.00027056944
	0.00095202215
	6.872898e-05
	0.0021645825
	7.251305e-05
	0.00023142833
	0.00022225524
	0.00027056944
	0

	0.000688801
	3.2787768e-05
	0.000140009
	0.0003338553
	0.0001914481
	6.520819e-05
	2.5393747e-05
	0.000688801
	3.2787768e-05
	0.000140009
	0.0003338553
	0.0001914481
	6.520819e-05
	2.5393747e-05
	0.000688801
	3.2787768e-05
	0.000140009
	0.0003338553
	0.0001914481
	6.520819e-05
	2.5393747e-05
	0.000688801
	3.2787768e-05
	0.000140009
	0.0003338553
	0.0001914481
	6.520819e-05
	2.5393747e-05
	0.000688801
	3.2787768e-05
	0.000140009
	0.0003338553
	0.0001914481
	6.520819e-05
	2.5393747e-05
	0.000688801
	3.2787768e-05
	0.000140009
	0.0003338553
	0.0001914481
	6.520819e-05
	2.5393747e-05
	0.000688801
	3.2787768e-05
	0.000140009
	0.0003338553
	0.0001914481
	6.520819e-05
	2.5393747e-05
	0.010248099
	7.870425e-05
	0.00017321217
	0.0006432618
	0.0006790752
	0.0035662162
	0.0001163021
	0.010248099
	7.870425e-05
	0.00017321217
	0.0006432618
	0.0006790752
	0.0035662162
	0.0001163021
	0.010248099
	7.870425e-05
	0.00017321217
	0.0006432618
	0.0006790752
	0.0035662162
	0.0001163021
	0.010248099
	7.8704

dict

In [18]:
'''#for key, value in results.items():
for key, value in range(sheet.nrows):
    #feature_array=results[key]
    #print("%s"%key)
    #print(feature_array)
    #for value in feature_array:
    print("\t%s:%s\n"%(key,value))
   ''' 

'#for key, value in results.items():\nfor key, value in range(sheet.nrows):\n    #feature_array=results[key]\n    #print("%s"%key)\n    #print(feature_array)\n    #for value in feature_array:\n    print("\t%s:%s\n"%(key,value))\n   '

In [19]:
#type(results)
#type(feature_array)
'''
print(results)
print("#############")
print(results[key])
print("#############")
print(feature_array)
'''

'\nprint(results)\nprint("#############")\nprint(results[key])\nprint("#############")\nprint(feature_array)\n'

In [39]:
import csv
with open('TFND_toxicity_features.csv', 'w') as f:
    for key in results.items():
        for key, value in results.items():
            #/f.write("\t%s:%s\n"%(key,value))
            #f.write("\t%s"%value)
            #/f.write("%s\t%s"%(key,dataset_cbow[key]))
       # f.write("\n
            print("\t%s"%value)
        print("Creating Toxicity dataset . . .")

	0.018986389
	0.00021364134
	0.00054498954
	0.0006309728
	0.00047218218
	0.017216811
	0.00026660215
Creating Toxicity dataset . . .
	0.018986389
	0.00021364134
	0.00054498954
	0.0006309728
	0.00047218218
	0.017216811
	0.00026660215
Creating Toxicity dataset . . .
	0.018986389
	0.00021364134
	0.00054498954
	0.0006309728
	0.00047218218
	0.017216811
	0.00026660215
Creating Toxicity dataset . . .
	0.018986389
	0.00021364134
	0.00054498954
	0.0006309728
	0.00047218218
	0.017216811
	0.00026660215
Creating Toxicity dataset . . .
	0.018986389
	0.00021364134
	0.00054498954
	0.0006309728
	0.00047218218
	0.017216811
	0.00026660215
Creating Toxicity dataset . . .
	0.018986389
	0.00021364134
	0.00054498954
	0.0006309728
	0.00047218218
	0.017216811
	0.00026660215
Creating Toxicity dataset . . .
	0.018986389
	0.00021364134
	0.00054498954
	0.0006309728
	0.00047218218
	0.017216811
	0.00026660215
Creating Toxicity dataset . . .


In [33]:
print(results)

{'toxicity': 0.018986389, 'severe_toxicity': 0.00021364134, 'obscene': 0.00054498954, 'identity_attack': 0.0006309728, 'insult': 0.00047218218, 'threat': 0.017216811, 'sexual_explicit': 0.00026660215}


In [32]:
x=results.values()
print(x)

dict_values([0.018986389, 0.00021364134, 0.00054498954, 0.0006309728, 0.00047218218, 0.017216811, 0.00026660215])


In [23]:
'''for key in results.items():
    feature_array=results[key]
    print("%s"%key)
    for value in feature_array:
        print("%s\t%s"%(key,feature_array[key]))
'''

'for key in results.items():\n    feature_array=results[key]\n    print("%s"%key)\n    for value in feature_array:\n        print("%s\t%s"%(key,feature_array[key]))\n'